In [1]:
# google driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# ディレクトリを移動
%cd "drive/My Drive/Colab Notebooks"
!ls

/content/drive/My Drive/Colab Notebooks
chatbot.ipynb  nucc.txt


In [4]:
# mecabをインストール
!apt-get install mecab
!pip install mecab-python3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmecab2 mecab-jumandic mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  libmecab2 mecab mecab-jumandic mecab-jumandic-utf8 mecab-utils
0 upgraded, 5 newly installed, 0 to remove and 25 not upgraded.
Need to get 16.5 MB of archives.
After this operation, 219 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-utils amd64 0.996-5 [4,856 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-jumandic-utf8 all 7.0-20130310-4 [16.2 MB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-jumandic all 7.0-20130310-4 [2,212 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab amd64 0.996-5 [132 kB]
Fetched 16.5 MB in 3

In [6]:
# 明大の会話コーパスを取得
!wget http://mmsrv.ninjal.ac.jp/nucc/nucc.zip
!unzip -o nucc.zip
!rm nucc.zip*

import os
import re
import MeCab

# 入力ファイル
nucc_dir = 'nucc'
files = os.listdir(nucc_dir)

# MeCab
mecab = MeCab.Tagger('-Owakati')

# 書き込みファイル
with open(nucc_dir + '.txt', 'w', encoding='utf-8') as fout:

  # ファイルを読み込む
  for file in files:
    fout.write('### {}\n'.format(file))

    with open(os.path.join(nucc_dir, file), 'r', encoding='utf-8') as fin:

      speaker = None
      sentence = ''

      for line in fin:

        # 文末の改行を除く
        line = line.strip()

        # @始まりはファイルの説明なので無視
        if re.search(r'＠', line):
          continue
        else:
          # 話し手は行の先頭
          m = re.match(r'(％.{3}|Ｘ|F.{3}|M.{3})：(.*)', line)

          if m:
            # 話し手が変わったとき
            if speaker and speaker != m.group(1):
              # %は会話に関係ないので除く、会話内容がある場合のみ追加
              if not re.search(r'％', speaker) and len(sentence) > 0:
                fout.write(sentence + '\n')

            # 話し手と会話内容を初期化
            speaker = m.group(1)
            sentence = ''
            line = m.group(2)
          
          # 整形(＊3つはおそらく固有名詞、＊で囲われているものは不明)
          line = line.replace('＊＊＊', '○○○').replace('＊', '')
          line = re.sub(r'(（|＜).+(）|＞)', '', line)
          # 分かち書き済みのテキストを追加
          if len(line) > 0:
            # 改行コードの削除、固有名詞はつなげておく
            line = mecab.parse(line).replace('\n', ' ').replace('○ ○ ○', '○○○')
            sentence += line

--2020-03-11 14:45:43--  http://mmsrv.ninjal.ac.jp/nucc/nucc.zip
Resolving mmsrv.ninjal.ac.jp (mmsrv.ninjal.ac.jp)... 202.245.100.170
Connecting to mmsrv.ninjal.ac.jp (mmsrv.ninjal.ac.jp)|202.245.100.170|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://mmsrv.ninjal.ac.jp/nucc/nucc.zip [following]
--2020-03-11 14:45:43--  https://mmsrv.ninjal.ac.jp/nucc/nucc.zip
Connecting to mmsrv.ninjal.ac.jp (mmsrv.ninjal.ac.jp)|202.245.100.170|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2187379 (2.1M) [application/zip]
Saving to: ‘nucc.zip’

nucc.zip            100%[===================>]   2.09M  4.35MB/s    in 0.5s    

2020-03-11 14:45:44 (4.35 MB/s) - ‘nucc.zip’ saved [2187379/2187379]

Archive:  nucc.zip
  inflating: nucc/data001.txt        
  inflating: nucc/data002.txt        
  inflating: nucc/data003.txt        
  inflating: nucc/data004.txt        
  inflating: nucc/data005.txt        
  inflating: nucc/data006.txt        
 